In [31]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

from dotenv import load_dotenv

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [12]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [13]:
## Checking the Models available 
import google.generativeai as gemini

## Configuring the GEMINI API key for accessing the model
gemini.configure(api_key=GEMINI_API_KEY)

for model in gemini.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004
models/aqa


In [14]:
## Creating PDF loader
def pdf_data_loader(input_pdf_file):
    reader = PyPDFLoader(input_pdf_file)
    return reader.load()

In [24]:
## Creating method for calling Google Gemini API

llm_model = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY, 
    model="models/gemini-1.5-pro-latest", 
    temperature=0.6, 
    top_p=0.8,
    convert_system_message_to_human=True
)
   

In [18]:
text = pdf_data_loader("D:\\manav_docs\\Manav_Garg_Resume_3.pdf")
resume_content = text[0].page_content

In [19]:
job_description = """
Master’s in a highly quantitative field (Machine Learning, Operational Research, Statistics, Mathematics, etc.) preferably from a reputable institution like IITs, IISc, or equivalent.
4+ years of industry experience in a data scientist role
Solid understanding of the mathematics related to data science - probability, statistics, linear algebra etc.
Extensive knowledge of various machine learning algorithms and concepts used to build predictive models
In-depth knowledge in at least 2 of the following data science domains:
Text Analysis and Natural Language Processing
Neural Networks and Deep Learning
Signal Processing, Image/Video processing and recognition
Reinforcement Learning
Hands on experience with computational packages (Numpy, Scipy, Pandas, Scikit-Learn, StatsModels, Keras, Tensorflow, etc.) and visualization tools
Highly proficient in SQL and good knowledge of relational database
Prior experience in deploying machine learning models into production
Experience in working with GPUs to develop models
Possess strong interpersonal and stakeholder management skills
Past and current experience writing and speaking about complex technical concepts to broad audiences in a simplified format
Strong written and verbal communication skills
Demonstrated the ability to independently develop new concepts and stay current with academic research
Exposure to SAP modules along with prior experience in cloud technologies like BTP, AWS, GCP, etc. is desired.
Publications or presentations in recognized Machine Learning and Data Mining journals/conferences
Identify and translate real world customer problems into feasible technical solutions and implement them independently. 
Expectations and Tasks:
Conceptualize, design and deliver high quality solutions and insightful analysis
Work on gathering data and requirements, participate in solution scoping and architecture, research and prototyping innovative models
Build solutions by exploring data, building & validating models and deploying them to deliver maximum business impact
Interact with both - internal and external stakeholders on a daily basis
Handle multiple projects as an individual contributor and be a multiplier by actively mentoring other resources within the team
Be up-to-date with latest and best practices in delivering enterprise grade data science solutions
Build credibility and trust with customers and project teams during project execution
Demonstrate adaptability and flexibility to manage deadline pressure, ambiguity and change
Continually enhance existing skills and seek new areas for personal development.
Collaborate with product owner, architect, fellow developers, quality and user assistance specialists on design, architecture, hands-on coding, testing, etc. to deliver best solutions to our customers
Work on the latest technology and ensure on-time, high quality solutions
Embrace lean and agile software development principles
Perform high-level analysis of any new requirements/change requests to the solution from a technical standpoint
Analyze the impact of any new requirements on the existing solution
Work closely with other CIS Engineering team members, SAP Product Development teams, external partners and customer teams to ensure successful project completion
"""

In [59]:
input_prompt="""
Hey Act Like a skilled or very experience ATS(Application Tracking System)
with a deep understanding of tech field,software engineering,data science ,data analyst
and big data engineer. Your task is to evaluate the resume based on the given job description.
You must consider the job market is very competitive and you should provide 
best assistance for improving the resumes. And also if possible tell whether the resume is applicable for the passed job description like applicable or not applicable.
Assign the percentage Matching based 
on Jd and the missing keywords with high accuracy
resume:{resume_content}
description:{job_description}

I want the response in one single string having the structure
{{"JD Match":"%", "MissingKeywords:[]", "Profile Summary":"", "applicable": ""}}
"""

In [60]:
prompt = PromptTemplate(
    input_variables=["resume_content", "job_description"],
    template=input_prompt
)

In [61]:
a = {
    "resume_content": resume_content,
    "job_description": job_description
}

In [69]:
chain = LLMChain(llm=llm_model, prompt=prompt)

In [71]:
response = chain.invoke(a)

In [75]:
response

{'resume_content': 'Manav Garg  \n(+91) 9821334081 | manavgarg026@gmail.com  | linkedin.com/in/manavgarg026  | https://github.com/Manav446  \nSummary:  \nHighly analytical, meticulously organized software developer skilled in software development, AI, and Data Science. \nKnown for collaborative teamwork, a strong sense of ownership, and a proven history of delivering top -tier strategic \nsolutions to overcome o bstacles and drive business advancement.  \nTech nical Skills : \n• Programming Languages : Python, Java  \n• Frameworks & Tools: TensorFlow, PyTorch, Langchain, LlamaIndex, Flask, Spring Boot, Spring MVC, Hibernate , \nSpring Batch , Microservices  \n• Generative AI Technologies: Open Source and paid LLM models (like Open AI,  Google Gemini, Mistal, Llama2, \nMistral etc.) , RAG  \n• Databases: PostgreSQL, MySQL, Microsoft SQL Server, Mongo -DB, Oracle  \n• Vector Databases: Chroma DB, Pinecone, Weaviate  \n• AI/ML Techniques: NLP, Neural Network optimization, Deep Learning, S